In [4]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas




In [5]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [6]:

"""pd.read_csv loads the input file and first line is col names (header = 0).
We want to remove the HTML markup tokens from the movie reviews we have with BeautifulSoup"""

train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header= 0, \
            delimiter="\t", quoting=3)

train.shape

train.columns.values
#shows first movie review in the col- "review"
print (train["review"][0])


from bs4 import BeautifulSoup
example1 = BeautifulSoup(train["review"][0])

print(example1.get_text())





FileNotFoundError: [Errno 2] No such file or directory: '../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip'

In [ ]:
"""Dealing with punct and stopwords. We want only the letters in the input file, and we 
want them to be lowercase. Stop words like 'the' are not useful for determining sentiment,
so we want to remove those from the text as well."""


import re
import nltk



letters_only = re.sub("[^a-zA-z]",         #pattern to search for 
                      " ",                 #pattern to replace with 
                      example1.get_text()) #text to search
                      
                      
                     
#print(letters_only)
#convert reviews to lower case and split them into individual words 
lower_case = letters_only.lower()
words = lower_case.split()

nltk.download()
from nltk.corpus import stopwords   #import stopwords
#now, deal with frequently occurring words that don't carry much meaning (stop words)
#print(stopwords.words("english"))


words = [w for w in words if not w in stopwords.words("english")]
#removes all stopwords 


#print(words)

In [ ]:
"""The function review_to_words puts everything we've done so far in a function.
It's input is a string with the raw movie review input, and the output is also a string which 
was been processed with only letters, and no HTML markup, or stopwords. We add this to 
clean_review_list and use the function for all 25000 movie reviews"""

def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))  



clean_review = review_to_words(train["review"][0])

num_reviews = train["review"].size
clean_review_list = []


for i in range(0, num_reviews):
    
    #print message if review divisible by 1000
    if ((i+1) % 1000 == 0):
        print ("Review %d of %d\n" % ( i+1, num_reviews ))     
    clean_review_list.append(review_to_words(train["review"][i]))
    
    

In [ ]:
"""In order to represent the frequency of words numerically, we represent all the words in the 
movie reviews dictionary as a 'Bag of words'. Then, we use fit_transform() to fit the model
we have and understand the word dictionary, and then represent these words as feature vectors."""

from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.

vectorizer = CountVectorizer(analyzer = "word", \
                             tokenizer = None,
                             preprocessor = None, \
                             stop_words = None, \
                             max_features = 5000)




train_data_features = vectorizer.fit_transform(clean_review_list)

#convert result to array
train_data_features = train_data_features.toarray()

print(train_data_features.shape)


vocab = vectorizer.get_feature_names()
print(vocab)


In [ ]:
"""Here, we simply count the frequency of words in the word dictionary by using np.sum() and print"""

import numpy as np 

dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)




In [ ]:
#supervised learning

from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100)

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
forest = forest.fit(train_data_features, train["sentiment"])

#now, we want to run our trained model on the test set 
test = pd.read_csv('..input/word2vec-nlp-tutorial/testData.tsv.zip', header=0, delimiter="\t", \
                   quoting=3)

print(test.shape)


"""# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] """

